In [3]:
# Databricks notebook source

import pyspark
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession, Row,Column
from pyspark.sql import HiveContext,SQLContext
from pyspark.sql import types as T
from pyspark.sql import functions as F



# from pyspark.sql.types import StructField, StringType, IntegerType, StructType, BooleanType
# from pyspark.sql.functions import (countDistinct, 
#                                    avg, 
#                                    stddev,
#                                    initcap,
#                                    abs,
#                                    min, 
#                                    max, 
#                                    mean, 
#                                    format_number,
#                                    dayofmonth,
#                                     hour,
#                                    dayofyear,
#                                    month,                                                 
#                                    year,
#                                    weekofyear,
#                                    format_number,
#                                    date_add,date_sub,date_trunc,datediff,add_months,last_day,current_date,
#                                    lower,
#                                    upper,
#                                    udf,
#                                    col,
#                                    when,
#                                    shuffle,
#                                    array,
#                                    round,
#                                    to_date,
#                                    udf,
#                                    date_format)

In [ ]:
# PySpark module needed to update the spark Dataframe

- SQLContext
- HiveContext
- functions from pyspark sql

# Connect to Hive Database
    # HiveContext.sql('use <database>')
    
# Read table from hive

    # df = hive Context.sql
    # df.show()
    
# check null in column an replace with 0
    
    # df = df.withcolumn('newcol', F.when(F.col('col1').isNull(),0).otherwise(F.col('col1')))

# Use equality to update column values

    # df = df.withColumn('newcol', F.when(F.col('col1')==0, 100).otherwise(F.col('col1')))

In [8]:

spark = SparkSession.builder.appName('pepdep').getOrCreate()


In [ ]:
                                #### two test files to join

In [ ]:
df_pep = spark.read.csv("/Users/naderhashemi/jose_pyspark/sparkFiles/pep.csv", inferSchema=True, header=True)


In [ ]:
df_dept = spark.read.csv('/Users/naderhashemi/jose_pyspark/sparkFiles/dept.csv', inferSchema=True, header=True)

In [ ]:
for i,j in enumerate(df_pep.columns):
    print(i,'-',j)

In [ ]:
df_dept.printSchema()

In [ ]:
df_pep.show()

In [ ]:
                                    
# df = df1.join(df2, on=['key'], how='inner')

join_df = df_pep.join(df_dept, on=['id'], how='left')
type(join_df)


In [ ]:
# Update column 'income' to be all positive number
# Update column 'lastName' to be proper case


inc_df = join_df.withColumn('income',(F.abs(join_df['income']))) \
                .withColumn('lastName',(F.initcap(join_df['lastName'])))

In [ ]:
inc_df.createOrReplaceTempView('j_sql')
spark.sql("select dep, mean(income) as avg_income from j_sql group by 1 order by dep").show()



In [ ]:
# create new column 'hi_income' that shows if income of each employee is greater or less than average of the 
# each group of the department
spark.sql("select concat(firstName,' ',lastName) as fullName \
          ,a.income \
          ,b.avg_income \
          ,case when a.income > b.avg_income then 'above_avg' \
                else 'below_avg' \
                end as hi_income \
          from j_sql as a \
          left join \
          (select dep, mean(income) as avg_income \
           from j_sql group by 1 order by dep) as b \
          on a.dep=b.dep").show()


In [ ]:
                                        # DATES in PySpark
    
#       to_date() will convert a string date type to a date type 
#       to_date function by default will add days as '01' if day is missing

    

In [ ]:
# spark = SparkSession.builder.appName('dds_files').getOrCreate()

In [ ]:
dds = spark.read.csv("/Users/naderhashemi/jose_pyspark/sparkFiles/DSS_by_Month_CY_2012_2021.csv" \
                     , inferSchema=True, header=True)

In [ ]:
dds.show()

In [ ]:
dds.select(F.countDistinct("program")).show()

In [ ]:
zeroDF = dds.withColumn('zerorow',F.when(F.col("zeroCol").isNull(),0) \
                       .otherwise(F.col("zeroCol"))).drop('zeroCol')

In [ ]:
zeroDF.show()

In [ ]:
# Aggregate 'active_recipients' column by grouping 'program' and 'year_month'

count_df=zeroDF.groupBy('program','year_month').sum('zerorow','active_recipients')

In [ ]:
# rename colums

renamecol = count_df.withColumnRenamed('sum(zerorow)','totZero') \
                    .withColumnRenamed('sum(active_recipients)','totActRec')

In [ ]:
# convert 'year_month' column to date type:  
agg_df = renamecol.select('*',F.to_date(count_df['year_month'],'yyyy-MM').alias('yearmon')).drop('year_month')

In [ ]:
agg_df.show(10)

In [ ]:
# using filter as a where clause in SQL

agg_df.filter(agg_df['totZero'] != 0).select('*')

In [ ]:
# counting all non zero 'sum(zerorow) that are not zero'

agg_df.filter(agg_df['totZero'] != 0).count()

In [ ]:
class nec():
       
    def __init__(self,endOfMonth,begOfMonth,today,dob):
        self.endOfMonth = endOfMonth 
        self.begOfMonth = begOfMonth
        self.today = today
        self.dob = dob
    
    @classmethod
    def lastDayofMonth(self,col):
        self.col = col
        return F.last_day(self.col)
  
    @classmethod
    def cap(self,cap):
        self.cap = cap
        return F.initcap(self.cap)
  
    @classmethod
    def age(self,x,y):
        self.x = x
        self.y = y
        z = self.x - self.y
        if z <= 9999 or z =='':
            return 1
        else:
            return int(str(x-y)[:-4])
       
         
      
    @classmethod
    def firstTwoDigit(self,p):
        self.p = p
        return (str(self.p)[:-4])

    @classmethod
    def noSpace(self,f):
        self.f = f
        n = ''
        for i in (self.f) or '':
            
            if i != ' ':
                n += i
        return n.upper()
    
    @classmethod
    def comstr(self,x):
        self.x = x
        z = ''
        for i in self.x or '':
            if i.isalpha():
                z += i
            
        return z.upper()
    

In [ ]:
# sqlContext.udf.register('firstTwoDigit', nec.firstTwoDigit)
firstTwoDig = F.udf(lambda z: nec.firstTwoDigit(z),T.StringType())

In [ ]:
NoSpace = F.udf(lambda x: nec.noSpace(x),T.StringType())

In [ ]:
NewAge = F.udf(lambda x,y: nec.age(x,y),T.IntegerType())

In [ ]:
nec.age(20210410,19590527)

In [ ]:
st='no shit !'
nec.noSpace(st)

In [ ]:
replace = F.udf(lambda t: nec.comstr(t),T.StringType())

In [ ]:
nec.comstr('997sh\!*##it w98o\rks !')

In [ ]:
agg_df.select(replace('program'),'*').show()

In [ ]:
agg_df.select(NoSpace('program'),'*').show()

In [ ]:
nec.age(30000,10000)

In [ ]:
nec.firstTwoDigit(450000)

In [ ]:
agg_df.select('*',nec.lastDayofMonth('yearmon')).show(3)

In [ ]:
# remove spaces from 'program' column by udf function 
spark.udf.register("noS", nec.noSpace,T.StringType())

In [ ]:
agg_df.createOrReplaceTempView("nsql")

In [ ]:
test=spark.sql("select * ,noS(program) as programNoSpace from nsql").show()

In [ ]:
                                # Date functions

In [ ]:
#            date_truc converts date to datetime type

date_df = agg_df.select('*',F.date_trunc('month','yearmon')).show()

In [ ]:
# datediff() function returns number of days from two given dates

age_df = agg_df.withColumn('daysBetweenTwoDates',(F.datediff(F.current_date(),agg_df['yearmon']))) \
               .select('*',F.current_date())

In [ ]:
#******       date_format() and cast() functions is used to convert a date to an integer **********************

age_days = age_df.withColumn('today',(F.date_format(F.current_date(),'yyyyMMdd')).cast(T.IntegerType())) \
                             .withColumn('yearmonday',F.date_format('yearmon','yyyyMMdd').cast(T.IntegerType())) \
                             .select('*')

In [ ]:
age_days.show()

In [ ]:
age_days.printSchema()

In [ ]:
age = age_days.withColumn('age',NewAge('today','yearmonday').alias('age')).select('*')

In [ ]:
age.show()
age.printSchema()

In [1]:
# This will sort alphabetically : capitalization IS First and matters

x = ['Python', 'programming', 'is', 'awesome!']
print(sorted(x))


['Python', 'awesome!', 'is', 'programming']


In [2]:
# This will Sort based on the given key function, Key can be any function and Capitalization is NOT matter

y = 'whatAnAssHOLe'
print(sorted(y, key=lambda arg: arg.lower()))


['a', 'A', 'A', 'e', 'h', 'H', 'L', 'n', 'O', 's', 's', 't', 'w']


In [ ]:

z = ['grduate', 'dORO', 'Si']
print(sorted(z, key=lambda arg: len(arg), reverse=False))

In [ ]:
# reduce() uses the function called to reduce the iterable to a single value:

from functools import reduce


x = ['Python', 'programming', 'is', 'awesome!', 'more of the same']

print(reduce(lambda val1,val2: val1+val2 , x))


In [ ]:
# data coming from twitter to Kinesis or Kafka, Flume goes to pyspark

In [15]:
data_schema = [T.StructField('year_month', T.StringType(), True),
               T.StructField('program', T.StringType(), True),
               T.StructField('active_recipients', T.StringType(), True),
               T.StructField('zeroCol', T.StringType(), True)
              ]

In [ ]:
dft = spark.read.csv("/Users/naderhashemi/jose_pyspark/sparkFiles/DSS_by_Month_CY_2012_2021.csv", schema=final_struct, header=True)

In [ ]:
dft.show()

In [ ]:
# pip install PyArrow

In [ ]:
import numpy as np
import pandas as pd

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Generate a pandas DataFrame
# pdf = pd.DataFrame(np.random.rand(100, 3))

# Create a Spark DataFrame from a pandas DataFrame using Arrow
df = spark.createDataFrame(mytext)

# Convert the Spark DataFrame back to a pandas DataFrame using Arrow
# result_pdf = df.select("*").toPandas()

In [ ]:
df.printSchema()

In [ ]:
df.show()